# Week 2 - Connect Tiki Scraper to PostgreSQL database

## Introduction

PostgreSQL is a free and open-source relational database management system. Using a database instead of regular spreadsheet will allow us to store a larger amount of data as well as offer a much higher capacity to later process our data. 

Instead of exporting the scraped data to a dataframe like we did in Week 1, this time we will push our data to a PostgreSQL database. 

PostgreSQL databases run locally and therefore each of us will need to install and create our own PostgreSQL database.

## Submission Guide
- Create a copy of this Jupyter Notebook
- Create a Github repository for your project
- Upload the Jupyter Notebook with your solution to your Github repository
- Submit your works through this [Google Form](https://forms.gle/6cgzgw4uKTTRZTri7).


# Your tasks: Replace "__" with the correct code

## Update the Scraper

Besides `pandas`, `requests` and `BeautifulSoup`, we will be using another library called `psycopg2` to interact with our PostgreSQL database.

In [111]:
# Import libraries
import pandas as pd
import requests
import psycopg2
from bs4 import BeautifulSoup

In [112]:
# Parser function to retrieve and parse the HTML code of a website 
def parser(url):
    
    # Plain HTML code
    plain = requests.get(url).text
    
    # Parser
    s = BeautifulSoup(plain, "html.parser")
    
    return s

In [113]:
# Function to get all URLs of categories on Tiki
def get_urls():
    
    url = 'https://tiki.vn/'
          
    # Run Parser on Tiki
    s = parser(url)
    
    # Initialize an empty list of category 
    category_list = []

    # Scrape
    
    # through the navigator bar on Tiki homepage
    for i in s.findAll('a',{'class':'MenuItem__MenuLink-tii3xq-1 efuIbv'}):
        
        # Get the category value
        category = i.find('span',{'class':'text'}).text 
        
        # Get the url value
        url = i['href'] + "&page=1"
         
        # Add category and url values to list
        category_list.append((category, url))
        
    return category_list

## Connect to PostgreSQL Database

The function `get_connection()` creates a connection to our local PostgreSQL database. 

Change the arguments of `psycopg2.connect()` with your information.

In [114]:
# Function to create connection to our PostgreSQL database
def get_connection():
    connection = psycopg2.connect(user = "postgres",
                                  password = 'anhhong2002',
                                  host = "127.0.0.1",
                                  port = "5432",
                                  database = "tiki")
    return connection

### _Test the function_

In [115]:
get_connection()

<connection object at 0x000001F45BD5F990; dsn: 'user=postgres password=xxx dbname=tiki host=127.0.0.1 port=5432', closed: 0>

## Create new table in PostgreSQL Database

`create_products_table()` creates an empty table in our local PostgreSQL database with predefined columns:
* product_id: varchar (20)
* product_seller_id: varchar (20)
* title: text
* price: integer
* img_url: text
* category: text

The `cursor` allows Python code to execute PostgreSQL command during our session (connection). `cursor` is always bound to our `connection`. 

In [198]:
# Function to create an empty `products` table in our database if it doesn't exist yet
def create_products_table():
    
    print('INFO create_products_table(): Create table products')
    
    try:
        # Create connection & cursor
        connection = get_connection()
        cursor = connection.cursor()
        
        # Execute the query
        cursor.execute('''CREATE TABLE IF NOT EXISTS products (
                            product_id INTEGER PRIMARY KEY,
                            product_seller_id TEXT,
                            title TEXT,
                            price TEXT,
                            img_url TEXT,                           
                            tikinow TEXT,
                            rating TEXT,
                            review TEXT,
                            category TEXT
                            );''')
        
        # Commit the changes made to our database
        connection.commit()

    except (Exception, psycopg2.Error) as error :
        print ("ERROR create_products_table(): Error while connecting to PostgreSQL", error)
        
        # In case of error, cancel all changes made to our database during the connection
        connection.rollback()
        return
    
    finally:
    
        # Close the connection & cursor
        cursor.close
        connection.close

### _Test the function_

In [199]:
create_products_table()

INFO create_products_table(): Create table products


## Insert data to PostSQL Database

`insert_data_to_db` function inserts the data row into our PostgreSQL table. It takes two arguments: `data` being the results the scrape() function and `table_name` being the table in our database that we want to insert `data` to.

This function replaces the step in `scrape_all()` where we add the result of `scrape(cat, url)` to `results`. See changes in funciton `scrape_all()` below.

In [210]:
# Function to insert new data to our table
def insert_data_to_db(data, table_name):
    
    print('INFO insert_data_to_db(): Insert data to DB')
    
    try:
        # Create connection & cursor 
        connection = get_connection()
        cursor = connection.cursor()
        
        # For each row (product) in data (product page),...
        for row in data:
            cursor.execute('SELECT 1 FROM products WHERE product_id = ' + row[0]);
            check_exist = cursor.fetchall()
            
            if len(check_exist) == 0:
                
                # Create the query to insert product information to 'table_name'  
                # Execute the query
                cursor.execute('INSERT INTO products VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)', row)

                # Commit the changes made to our database
                connection.commit()

    except (Exception, psycopg2.Error) as error :
        print ("ERROR save_data_to_db(): Error while connecting to PostgreSQL", error)
        
        # In case of error, cancel all changes made to our database during the connection
        connection.rollback()
        
    finally:
        
        # Close the connection & cursor
        cursor.close()
        connection.close()

To check if we have successfully created table `products`, go to your terminal, connect to your PostgreSQL database `psql -U [user_name] tiki` then enter `\d products`

`select_data_from_db()` will get all the data from table `products` to help us see if your scraper works.

In [184]:
# Function to get all data from table 'products'
def select_data_from_db():
    
    try:
        # Create connection & cursor 
        connection = get_connection()
        cursor = connection.cursor()
        
        # Get all data from table 'products'
        cursor.execute('SELECT * FROM products;')      
        
        # Fetch (Collect) the data and save them in 'data'
        data = cursor.fetchall()
        
        return data 

    except (Exception, psycopg2.Error) as error :
        print ("ERROR save_data_to_db(): Error while connecting to PostgreSQL", error)
        
        # In case of error, cancel all changes made to our database during the connection
        connection.rollback()
        
    finally:
        
        # Close the connection & cursor
        cursor.close()
        connection.close()

The `scrape(cat, url)` function is the same as the one we created in Week 1.

In [217]:
# Web scrape function
def scrape(cat, url):
    
    # Initialize empty 'results' list
    results = []
   
    # Run Parser on the product page
    s = parser(url)
    
    # Find all tags <div class='product-item'> and store them in 'prodct_items' list, each tag represent a product
    product_items = s.findAll('div',{'class':'product-item'})
    
   # if len(product_items[i].select('span.rating-content span[style]')) > 0:
    # rating = rating[0]["style"][len("width:"):]
    # If the tag list is empty (i.e. the page doesn't have any product), return an empty list.
    if len(product_items) == 0:
        return []

    # If the tag list is not empty (i.e. the page has products),...
    else:
        tiki_now = 0
        review_total = 0
        
        # Iterate through all product and store the product information in the 'row' list
        for i in range(len(product_items)):
            rating = 0
            if len(product_items[i].select(".rating-content span")) > 0:
                rating = product_items[i].select_one(".rating-content span")["style"].split(':')[1].replace("%", "")
            
            if len(product_items[i].findAll('i',{'class':'tikicon icon-tikinow'})) > 0:
                tiki_now = 1
            
            if len(product_items[i].findAll('div',{'class':'review-wrap'})) > 0:
                review_total = product_items[i].find('p',{'class':'review'}).string.replace(" nhận xét)","").replace("(","")
                
            row = [product_items[i]['data-id'],
                   product_items[i]['data-seller-product-id'], 
                   product_items[i]['data-title'],
                   product_items[i]['data-price'], 
                   product_items[i].find('img',{'class':'product-image img-responsive'})['src'],  
                   tiki_now,
                   rating,
                   review_total,
                   product_items[i]['data-category']]
            # Add the product information of each product into 'results' list
            results.append(row)
    # Return the list `results`   
    return results

In [191]:
scrape_all()

INFO scrape_all(): Start scraping
Scraping Voucher - Dịch Vụ - Thẻ Cào
INFO insert_data_to_db(): Insert data to DB
['21534194', '21534195', 'BA GÁC NƯỚNG & BIA - COMBO NƯỚNG BA GÁC ĐẶC BIỆT 6 MÓN DÀNH CHO 03 NGƯỜI', '320000', 'https://salt.tikicdn.com/cache/200x200/ts/product/72/6c/9e/7406664fe26402503c3933b911979d2c.jpg', 0, 0, 0, 'Voucher - Dịch vụ/Nhà hàng - Ăn uống/Nhà hàng']
Scraping Sách, VPP & Quà Tặng
INFO insert_data_to_db(): Insert data to DB
['436515', '2686829', "Harry Potter Boxed Set: The Complete Collection Children's (Paperback) - Bloomsbury UK Edition", '998000', 'https://salt.tikicdn.com/cache/200x200/ts/product/05/30/b0/b1ee828cc3341369acb765253a28901a.png', 1, '95', '433', 'Nhà Sách Tiki/English Books/Fiction - Literature/Science Fiction & Fantasy']
Scraping Hàng quốc tế
INFO insert_data_to_db(): Insert data to DB
['20088741', '20088742', '[PRE-ORDER] BTS WORLD (Original Soundtrack)+ Poster', '323000', 'https://salt.tikicdn.com/cache/200x200/ts/product/3f/18/72/760c

There are a few changes we will make to our `scrape_all()` function:
* Instead of initializing an empty `results` list, we will first run `create_products_table()` to create the `products` table in our database if it doesn't exist yet.
* Instead of add the result of `scrape(cat, url)` to `results`, we will add it directly to our table using `insert_data_to_db(data, table_name)`
* Since we have already inserted data to the `products` table while the function runs, we won't need to return the list `results`

In [215]:
# Run scrape fuction on every page

def scrape_all():
    print('INFO scrape_all(): Start scraping')
    
    # Initialize 'queue' list with the results of get_urls()
    queue = get_urls()
    
    # Create table `products` if it doesn't exist yet
    # results = []  <-- This is what we did in the first week
    results = []
    
    # While there are items in `queue`,...
    while len(queue) != 0:
        
      # `url` is set to the url of last item in `queue`
        url = queue[-1][-1]
      
      # `cat` is set to the category of last item in `queue`
        cat = queue[-1][0]
    
      # Remove the last item in queue
        queue = queue[:-1]

        print('Scraping', cat)

      # Run scrape(cat, url) with given `cat` and `url`
        new_rows = scrape(cat, url)

      # If the result of scrape(cat, url) is not an empty list (i.e. the page has products),...  
        if len(new_rows) != 0:
        
            # Insert the result of scrape(cat, url) to table `product`
            # results += new_rows  <-- This is what we did in the first week
            insert_data_to_db(new_rows, 'products')
            results += new_rows
            
            # Generate next page url 
            page = int(url[url.rfind("=")+1:])+1
            url = url[:url.rfind("=")+1] + str(page)
            
            print(url)
            
            # Use this to limit our scraper to scrape only the first 10 pages of each category
            # We do this to test our scraper with a smaller task first before running through every product page
            #if page < 10:
            
            # Add the new page url to the end of list `queue`
            queue.append((cat, url))
            
        print('Add next page', page)
    
    print('Task completed!')

### _Test the function_

In [218]:
scrape_all()

INFO scrape_all(): Start scraping
Scraping Voucher - Dịch Vụ - Thẻ Cào
INFO insert_data_to_db(): Insert data to DB
https://tiki.vn/voucher-dich-vu/c11312?src=c.11312.hamburger_menu_fly_out_banner&page=2
Add next page 2
Scraping Voucher - Dịch Vụ - Thẻ Cào
INFO insert_data_to_db(): Insert data to DB
https://tiki.vn/voucher-dich-vu/c11312?src=c.11312.hamburger_menu_fly_out_banner&page=3
Add next page 3
Scraping Voucher - Dịch Vụ - Thẻ Cào
INFO insert_data_to_db(): Insert data to DB
https://tiki.vn/voucher-dich-vu/c11312?src=c.11312.hamburger_menu_fly_out_banner&page=4
Add next page 4
Scraping Voucher - Dịch Vụ - Thẻ Cào
INFO insert_data_to_db(): Insert data to DB
https://tiki.vn/voucher-dich-vu/c11312?src=c.11312.hamburger_menu_fly_out_banner&page=5
Add next page 5
Scraping Voucher - Dịch Vụ - Thẻ Cào
INFO insert_data_to_db(): Insert data to DB
https://tiki.vn/voucher-dich-vu/c11312?src=c.11312.hamburger_menu_fly_out_banner&page=6
Add next page 6
Scraping Voucher - Dịch Vụ - Thẻ Cào
INFO

### _Test the results by getting all data in table `products` and put them in a `pandas` dataframe_

In [219]:
data = select_data_from_db()
df = pd.DataFrame(data, columns = ['product_id', 'seller_id', 'title', 'price', 'image_url', 'tikinow', 'rating', 'review', 'category'])
df

,product_id,seller_id,title,price,image_url,tikinow,rating,review,category
0,21534194,21534195,BA GÁC NƯỚNG & BIA - COMBO NƯỚNG BA GÁC ĐẶC BI...,320000,https://salt.tikicdn.com/cache/200x200/ts/prod...,0,0,0,Voucher - Dịch vụ/Nhà hàng - Ăn uống/Nhà hàng
1,23264624,23264630,Capella Park View - Buffet Chay Mùa Vu Lan Hơn...,189000,https://salt.tikicdn.com/cache/200x200/ts/prod...,0,0,0,Voucher - Dịch vụ/Nhà hàng - Ăn uống/Buffet/Bu...
2,16563156,16563157,Aka House - Đại Tiệc Buffet Nướng & Lẩu Nhật B...,229000,https://salt.tikicdn.com/cache/200x200/ts/prod...,0,0,0,Voucher - Dịch vụ/Nhà hàng - Ăn uống/Buffet/Bu...
3,20475731,20475732,"Kohaku Hotpot - Buffet Lẩu Nướng Hải Sản, Bò M...",279000,https://salt.tikicdn.com/cache/200x200/ts/prod...,0,100,1,Voucher - Dịch vụ/Nhà hàng - Ăn uống/Buffet/Bu...
4,7063331,7063333,CGV E-Voucher Vé Xem Phim 2D,88000,https://salt.tikicdn.com/cache/200x200/ts/prod...,0,80,143,Voucher - Dịch vụ/Sự kiện - Giải trí/Ca nhạc -...
5,11587220,11587221,Vé Tham Quan Bảo Tàng Tranh 3D Artinus (Quận 7...,89000,https://salt.tikicdn.com/cache/200x200/ts/prod...,0,83,11,Voucher - Dịch vụ/Sự kiện - Giải trí/Vui chơi ...
6,1559415,1559417,Phiếu Quà Tặng Tiki 100.000đ,100000,https://salt.tikicdn.com/cache/200x200/ts/prod...,0,98,163,Voucher - Dịch vụ/Phiếu quà tặng
7,1559411,1559413,Phiếu Quà Tặng Tiki 50.000đ,50000,https://salt.tikicdn.com/cache/200x200/ts/prod...,0,96,53,Voucher - Dịch vụ/Phiếu quà tặng
8,16056469,16056470,Vé Bảo Tàng Tranh 3D Art In Paradise Đà Nẵng,87500,https://salt.tikicdn.com/cache/200x200/ts/prod...,0,100,1,Voucher - Dịch vụ/Du lịch - Khách sạn/Vé tham ...
9,14615078,14615079,Lotte Cinema E-voucher Vé Xem Phim 2D - Áp Dụn...,69000,https://salt.tikicdn.com/cache/200x200/ts/prod...,0,0,1,Voucher - Dịch vụ/Sự kiện - Giải trí/Ca nhạc -...


Alternatively, you can connect to your PostgreSQL database `psql -U [user_name] tiki` and run query `SELECT * FROM products;`